University of Helsinki, Master's Programme in Data Science  
DATA20019 Trustworthy Machine Learning, Autumn 2021  
Antti Honkela and Ossi Räisä  

# Project 2: Real-life privacy-preserving machine learning

Deadline for returning the solutions: 28 November 23:55.

## General instructions (IMPORTANT!)

1. This is an individual project. You can discuss the solutions with other students, but everyone needs to write their own code and answers.
2. Please return your solutions as a notebook. When returning your solutions, please leave all output in the notebook.
3. When returning your solutions, please make sure the notebook can be run cleanly using "Cell" / "Run All".
4. Please make sure there are no dependencies between solutions to different problems.
5. Please make sure that your notebook will not depend on any local files.
6. Please make sure that the solutions for each problem in your notebook will produce the same results when run multiple times, i.e. remember to seed any random number generators you use (`numpy.random.seed()`, `torch.manual_seed()`!).


## Task 1: Differentially private logistic regression with DP-SGD

The Opacus (https://opacus.ai/) library provides implementations of many differentially private optimisation algorithms for deep learning and other models for the PyTorch (https://pytorch.org/) library. In order to perform these exercises, you will need to install Opacus, PyTorch and their dependencies according to instructions given on the websites. If you have not used PyTorch before, https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html is a good tutorial on the basics.

In order to study Opacus, we will use logistic regression on the UCI Adult data set (https://archive.ics.uci.edu/ml/datasets/Adult). (The data set is a standard benchmark data set that is available in various packages - feel free to use one of those.)  

A simple example implementation of the model is available at (https://www.cs.helsinki.fi/u/oraisa/tml/dp_log_reg_opacus.py).
The code has been adapted from tutorials provided with Opacus.

The definition of the logistic regression model binary classification is itself very straightforward in PyTorch, simply using a single fully connected linear layer with cross entropy loss:
```{python}
class LogisticRegression(nn.Module):
    def __init__(self, dim, bias=True):
        super().__init__()
        self.dim = dim
        # Define logistic regression using torch.nn layer and loss function
        self.linear = nn.Linear(dim, 1, bias=bias)
        self.loss = nn.BCEWithLogitsLoss(reduction="sum")
```
Training with privacy is done using the `PrivacyEngine` class, which is simply attached to 
any supported PyTorch optimiser.
```{python}
privacy_engine = PrivacyEngine(
    model,
    sample_rate=sample_rate,
    alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 200)),
    noise_multiplier=noise_multiplier,
    max_grad_norm=clip_bound,
    secure_rng=False, # Should set to True for production use
)
# Fix randomness, will throw an error with secure_rng=True
privacy_engine._set_seed(random_seed) 
privacy_engine.attach(optimizer)
```
Data subsampling is done by accessing the data through a `DataLoader` object that is configured with an Opacus-specific batch sampler.
```{python}
train_loader = DataLoader(
    train_tensor,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(train_tensor),
        sample_rate=sample_rate,
        # Should use a cryptographically secure PRNG in production
        generator=torch.Generator().manual_seed(random_seed), 
    ),
)
```
The `secure_rng` and `generator` parameters of `PrivacyEngine` and `UniformWithReplacementSampler` define the random number generators used for noise and minibatch sampling. In this exercise, we use the default RNGs of PyTorch, and manually seed them for reproducibility, but true DP guarantees require using a cryptographically secure RNG.

The `alphas` parameter of `PrivacyEngine` is related to Rényi differential privacy (RDP), which Opacus uses internally to compute privacy bounds. RDP is an alternative definition of DP, where privacy bounds are parameterised by an $(\alpha, \epsilon)$-pair. Opacus computes RDP bounds for the given $\alpha$ values, converts each RDP bound to an $(\epsilon, \delta)$-bound with $\delta$ given, and returns the smallest $\epsilon$, along with the corresponding $\alpha$.
This means that if the given list of $\alpha$ values does not contain the optimal $\alpha$, the returned $\epsilon$ values are too large. As a simple heuristic, if the returned $\alpha$ value is one of the bounds of the given list of $\alpha$ values, you should expand the list.

The rest of the example provides supporting architecture. The `generate_data` function generates a small test dataset, and `create_data_loaders` creates `DataLoader` objects for training and test data that can be passed to the `train` and `test` functions. Key parameters of the algorithm are defined at the end of the file and given as parameters to the `train` function. These include:
```{python}
# Learning rate for training
learning_rate = 0.05
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 2
# Clipping norm
clip_bound = 1
# Batch size as a fraction of full data size
sample_rate = 0.03
# Number of epochs
num_epochs = 2
```

`learning_rate` is the initial learning rate for the SGD optimiser. Larger value means faster learning but can cause instability.  
`noise_multiplier` controls the amount of noise added in DP-SGD: higher value means more noise. The value is defined relative to the gradient clip bound.  
`clip_bound` is the maximum norm at which per-example gradients are clipped. Smaller values mean less noise with the same level of privacy, but too small values can bias the results and make learning impossible.  
`sample_rate` is the fraction of full data used for each minibatch, which impacts privacy via amplification from subsampling. While a smaller sample rate increases privacy for a single update, it also increases the number of updates in a single epoch, which reduces privacy. The latter effect tends to be stronger than the first, so increasing sample rate increases privacy, but having too few updates limits the amount of learning.  
`num_epochs` controls the length of training as a number of passes over the entire data.

i. Test how these parameters (clip bound, sample rate, noise multiplier, learning rate and number of epochs) affect the accuracy of the classifier and its privacy. (You can use the test dataset from `generate_test_data` or the Adult dataset.)

ii. How accurate classifier can you build to predict if an individual has an income of at most 50k with the Adult dataset, using DP with $\epsilon=1, \delta = 10^{-5}$? Report your accuracy on a separate test set not used in learning. (Opacus does not offer an easy way to set the parameters to get $\epsilon = 1$, so you must tweak the parameters such that $\epsilon \leq 1$. You can use the function `check_privacy` to evaluate the privacy level at given parameters without actually running the optimisation.)

Hint: the data set includes many categorical variables. In order to use these, you will need to use a one-hot encoding with $n-1$ variables used to denote $n$ values so that $k$th value is represented by value 1 in $k-1$st variable and zeros otherwise. You do not need to use all of the variables for DP training, as a large number of variables increases the noise DP-SGD has to add, while some of the variables may not be useful for the prediction task.

Note: as noted at the lecture, testing several hyperparameters and choosing the best has an impact on the privacy guarantees.

Note: Unlike what the name suggests, `UniformWithReplacementSampler` samples a minibatch by selecting each element independently with the probability `sample_rate`. This implies that different minibatches will be of different sizes. This form of sampling works well with add/remove neighbourhoods that are commonly used in DP deep learning.

### Answer

#### Question I

For this question I used the random data generated by `generate_random_data()` function. I tested $\epsilon$ and accuracy on different parameters:

- learning_rate = [0.01, 0.05, 0.1, 0.5]
- noise_multiplier = [1, 1.5, 2, 2.5]
- clip_bound = [0.5, 1, 1.5, 2]
- sample_rate = [0.01, 0.03, 0.1, 0.2]

Then, I created a plot for each hyperparameter, relating accuracy score and $\epsilon$ on the number of epochs.

##### Preparatory code

In [ ]:
import sys
!{sys.executable} -m pip install torch opacus catboost

In [17]:
import numpy as np
import torch
import torch.nn as nn
import opacus
from torch.utils.data import DataLoader, TensorDataset
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

class LogisticRegression(nn.Module):
    def __init__(self, dim, bias=True):
        super().__init__()
        self.dim = dim
        self.linear = nn.Linear(dim, 1, bias=bias)
        self.loss = nn.BCEWithLogitsLoss(reduction="sum")
    
    def forward(self, x):
        return self.linear(x).view(-1)

    def train_step(self, batch):
        x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        return loss

    def test_step(self, batch):
        x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        preds = out > 0
        corrects = torch.tensor(torch.sum(preds == y).item())
        return loss, corrects

def train(model, train_loader, opt_func, learning_rate, num_epochs, sample_rate, noise_multiplier, clip_bound, delta, random_seed=474237, verbose=False):
    optimizer = opt_func(model.parameters(), learning_rate)
    privacy_engine = opacus.PrivacyEngine(
        model,
        sample_rate=sample_rate,
        alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 200)),
        noise_multiplier=noise_multiplier,
        max_grad_norm=clip_bound,
        secure_rng=False,
    )
    privacy_engine._set_seed(random_seed)
    privacy_engine.attach(optimizer)

    for epoch in range(num_epochs):
        losses = []
        for batch in train_loader:
            loss = model.train_step(batch)
            loss.backward()
            losses.append(loss)
            optimizer.step()
            optimizer.zero_grad()
        
        if verbose:
            print("Epoch {}, loss = {}".format(epoch + 1, torch.sum(loss) / len(train_loader)))

    epsilon, alpha = optimizer.privacy_engine.get_privacy_spent(delta)
    return epsilon, alpha

def test(model, test_loader):
    with torch.no_grad():
        losses = []
        accuracies = []
        total_size = 0
        for batch in test_loader:
            total_size += len(batch[1])
            loss, corrects = model.test_step(batch)
            losses.append(loss)
            accuracies.append(corrects)

        average_loss = np.array(loss).sum() / total_size
        total_accuracy = np.array(accuracies).sum() / total_size
        return average_loss, total_accuracy

def create_data_loaders(train_tensor, test_tensor, sample_rate, random_seed=4732842):
    train_loader = DataLoader(
        train_tensor,
        batch_sampler=UniformWithReplacementSampler(
            num_samples=len(train_tensor),
            sample_rate=sample_rate,
            generator=torch.Generator().manual_seed(random_seed),
        ),
    )
    test_loader = DataLoader(test_tensor, 64)
    return train_loader, test_loader

def check_privacy(sample_rate, noise_multiplier, num_epochs, delta = 1e-5,
                  alphas = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 200))):
    rdp = opacus.privacy_analysis.compute_rdp(sample_rate, noise_multiplier, int(1 / sample_rate) * num_epochs, alphas)
    epsilon, opt_order = opacus.privacy_analysis.get_privacy_spent(alphas, rdp, delta)
    return epsilon, opt_order

def generate_random_data():
    np.random.seed(4242)

    n_train = 2000
    n_test = 2000
    input_dim = 5

    N = n_train + n_test
    X0 = np.random.randn(N, input_dim)
    temp = X0 @ np.random.randn(input_dim, 1) + np.random.randn(N, 1)
    Y0 = np.round(1/(1+np.exp(-temp)))

    train_X = X0[0:n_train, :]
    test_X = X0[n_train:N, :]
    train_Y = Y0[0:n_train, 0]
    test_Y = Y0[n_train:N, 0]
    train_X = np.array(train_X, dtype=np.float32)
    test_X = np.array(test_X, dtype=np.float32)
    train_Y = np.array(train_Y, dtype=np.int32)
    test_Y = np.array(test_Y, dtype=np.int32)

    train_tensor = TensorDataset(torch.tensor(train_X), torch.tensor(train_Y, dtype=torch.float32))
    test_tensor = TensorDataset(torch.tensor(test_X), torch.tensor(test_Y, dtype=torch.float32))

    return train_tensor, test_tensor

##### Actual code

In [ ]:
# EXECUTION
torch.manual_seed(472368)
np.random.seed(0)
train_tensor, test_tensor = generate_random_data()
input_dim = train_tensor[0][0].size(dim=0)

delta = 1e-5

# Stock values
learning_rate = 0.05
noise_multiplier = 2
clip_bound = 1
sample_rate = 0.03
num_epochs = 2

# Experiments
learning_rates = [0.01, 0.05, 0.1, 0.5]
noise_multipliers = [1, 1.5, 2, 2.5]
clip_bounds = [0.5, 1, 1.5, 2]
sample_rates = [0.01, 0.03, 0.1, 0.2]

In [ ]:
# Test learning rates
for lr in learning_rates:
    x = []
    y = []
    train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

    for epoch in range(1,20):
        model = LogisticRegression(input_dim)
        epsilon, alpha = train(
            model, train_loader, torch.optim.SGD, lr, epoch, sample_rate,
            noise_multiplier, clip_bound, delta
        )
        x.append(epsilon)
        _, total_accuracy = test(model, test_loader)
        y.append(total_accuracy)
    plt.plot(x, y, label=lr)
plt.xlabel('epsilon')
plt.ylabel('accuracy')
plt.legend()
plt.show()

#### Question II

In [ ]:
import pandas as pd
from catboost.datasets import adult

# prepare adult dataset
train_set, test_set = adult()
data = train_set.append(test_set, ignore_index=False)
data = data.reset_index()
del data['index']

## Task 2: DP Image Classification

Using the above code as a basis, build a DP image classifier for the MNIST dataset. The dataset consists of 28x28 images of handwritten digits (0-9), and the task is to classify which digit each image represents. 

A simple implementation of a convolutional neural network for image classification is available (https://www.cs.helsinki.fi/u/oraisa/tml/pytorch_mnist.py), based on the Opacus MNIST tutorial. The example requires the Torchvision (https://pytorch.org/vision/stable/index.html) library, which you should install. The example contains the `SampleConvNet` class that can be used with the `train` and `test` functions from the last exercise, as well as the `get_mnist_dataset` function that loads the MNIST dataset. The dataset is downloaded the first time the function is called and is placed in a directory called `mnist`. By default, the function only loads half of the training data (30 000 images) to save computation time, but the number of training images can be given as a parameter.

i. Train the model using the hyperparameters given below and compute accuracy on test data.
```{python}
clip_bound = 1.5
learning_rate = 0.25
sample_rate = 0.004
noise_multiplier = 1.3
num_epochs = 5
```
The training may take a minute or two. You can pass `verbose=True` to the `train` function to get updates for every completed epoch.
You should get $\epsilon \approx 0.62$ with $\delta = 10^{-5}$ and an accuracy of 80-90%.

ii. Test the impact of the different DP parameters with the aim of creating maximally accurate classifier, given privacy bounds $\epsilon = 1$, $\delta = 10^{-5}$. You can also try changing the optimiser.

Note: Running the learning with many parameters can take a long time, so you should plan your work to keep the number of runs reasonable. The grading of part ii is based on obtaining an overall picture of the impact of key parameters, not the absolute accuracy obtained.

## Task 3: Your own problem in privacy-preserving machine learning

State and solve your own problem related to privacy-preserving machine learning.

You can use code available online, as long as you cite the source.

You can for example try reproducing the results of some interesting paper using their data or your own data, try out some of the privacy attacks, or simply try the above examples using more complex models and/or on different data sets.

If your problem is based on some previous problem, it should extend it in a non-trivial manner (not just running exact same code with new parameters or data).

The evaluation of the project will take the difficulty of your chosen problem into account.

This task is worth as much as two regular problems.